<a href="https://colab.research.google.com/github/sanath8107/strategies/blob/main/get_nse_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys

import requests
import csv
import datetime

cookie_url = 'https://www.nseindia.com/get-quotes/derivatives?symbol=BANKNIFTY'
equity_base_url = 'https://www.nseindia.com/api/historical/cm/equity'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) '
                         'Chrome/87.0.4280.88 Safari/537.36'}


def get_urls(scrip, from_date, to_date, instrument_type):
    url_list = []

    # temp_from_date = from_date
    from_date = datetime.datetime.strptime(from_date, '%d-%m-%Y')
    to_date = datetime.datetime.strptime(to_date, '%d-%m-%Y')

    day_diff = (to_date - from_date).days
    if day_diff < 0:
        print("From date should be earlier")
        sys.exit(0)
    print("Difference in days: ", day_diff)
    temp_date = from_date + datetime.timedelta(days=365 * 2)
    while temp_date <= to_date:
        equity_data_url = equity_base_url
        equity_data_url += '?' + 'symbol=' + scrip.upper()
        equity_data_url += '&' + 'series=' + '[%22' + instrument_type + '%22]'
        equity_data_url += '&' + 'from=' + from_date.strftime("%d-%m-%Y")
        equity_data_url += '&' + 'to=' + temp_date.strftime("%d-%m-%Y")
        equity_data_url += '&' + 'csv=true'
        url_list.append(equity_data_url)
        print(equity_data_url)
        from_date = temp_date
        temp_date = from_date + datetime.timedelta(days=365 * 2)

    equity_data_url = equity_base_url
    equity_data_url += '?' + 'symbol=' + scrip.upper()
    equity_data_url += '&' + 'series=' + '[%22' + instrument_type + '%22]'
    equity_data_url += '&' + 'from=' + from_date.strftime("%d-%m-%Y")
    equity_data_url += '&' + 'to=' + to_date.strftime("%d-%m-%Y")
    equity_data_url += '&' + 'csv=true'
    url_list.append(equity_data_url)
    print(equity_data_url)

    return url_list


def get_data(scrip, from_date, to_date, instrument_type):
    url_list = get_urls(scrip, from_date, to_date, instrument_type)
    full_data = []

    with requests.session() as s:
        # load cookies:
        s.get(cookie_url, headers=headers)

        for url in url_list:
            _data = s.get(url, headers=headers)  # .json()
            decoded_content = _data.content.decode('utf-8')
            cr = csv.reader(decoded_content.splitlines(), delimiter=',')
            my_list = list(cr)
            my_list.reverse()
            title_row = my_list.pop()
            for row in my_list:
                full_data.append(row)
    return full_data, title_row

# def backtest(data, strategy):
#     return result


data, title_row = get_data('MARUTI', '01-10-2017', '10-10-2020', 'EQ')
# result = backtest(data, macd)
# print(result.profit_percentage)
for row in data:
    print(row)
print("total number of days: ", len(data))